Bryan Flores <br>
ADS-509 <br>
Assignment 2.1 <br>
January 22, 2024

# ADS 509 Assignment 2.1: Tokenization, Normalization, Descriptive Statistics 

This notebook holds Assignment 2.1 for Module 2 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In the previous assignment you pulled lyrics data on two artists. In this assignment we explore this data set and a pull from the now-defunct Twitter API for the artists Cher and Robyn.  If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Canvas. 

This assignment asks you to write a short function to calculate some descriptive statistics on a piece of text. Then you are asked to find some interesting and unique statistics on your corpora. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import nltk
nltk.download('stopwords')  # Uncomment if necessary

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

sw = stopwords.words("english")

In [27]:
# Add any additional import statements you need here
from nltk.tokenize import word_tokenize
from pathlib import Path
import glob

In [4]:
# change `data_location` to the location of the folder on your machine.
# data_location = "/users/chandler/dropbox/teaching/repos/ads-tm-api-scrape/"
data_location = "data/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

In [5]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    num_tokens = len(tokens)
    num_unique_tokens = len(Counter(tokens).keys())
    lexical_diversity =  num_unique_tokens / num_tokens
    num_characters =  sum([len(token) for token in tokens])

    # Find 5 most common tokens and create dataframe for presentation
    top_5_tokens = Counter(tokens).most_common(5)
    df = pd.DataFrame(top_5_tokens, columns=['token', 'frequency'])
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        print("===========================")
        print(f"The 5 most common tokens: ")
        print(df)        
        
    return([num_tokens, 
            num_unique_tokens,
            lexical_diversity,
            num_characters
            ])
    

In [6]:
text = """here is some example text with other example text here in this text""".split()
assert(descriptive_stats(text, verbose=True)[0] == 13)
assert(descriptive_stats(text, verbose=False)[1] == 9)
assert(abs(descriptive_stats(text, verbose=False)[2] - 0.69) < 0.02)
assert(descriptive_stats(text, verbose=False)[3] == 55)


There are 13 tokens in the data.
There are 9 unique tokens in the data.
There are 55 characters in the data.
The lexical diversity is 0.692 in the data.
The 5 most common tokens: 
     token  frequency
0     text          3
1     here          2
2  example          2
3       is          1
4     some          1


Q: Why is it beneficial to use assertion statements in your code? 

A: Assertions allow you to test your logic for errors and invalid assumptions and explicitly state what we expect to be True or False.

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [80]:
# Read in the lyrics data
lyrics_path = data_location + lyrics_folder

""" 
    Example format:
    {
        'artist': cher,
        'filename': "cher_88degrees.txt",
        'title': "88 Degrees",
        'lyrics': "Stuck in L.A., ain't got no friends ..."
    }
"""

lyrics_list = []

for subdir in glob.iglob(f'{lyrics_path}/*'):

    # Define subdirectories
    PATH = Path(subdir).glob('*')

    # Iterate over song txt files
    for file in PATH:

        # Obtain file names, artists, song titles, and lyrics 
        filename = Path(file).name
        artist = filename.split("_")[0]
        title = open(file, 'r').readline().strip("\n")
        lyrics = ' '.join([line.strip() for line in open(file, 'r').readlines()[3:]]).lstrip()

        # Key-values
        lyrics_dict = {
            'artist': artist,
            'filename': filename,
            'title': title,
            'lyrics': lyrics
            }
        lyrics_list.append(lyrics_dict)

lyrics_df = pd.DataFrame(lyrics_list)

In [81]:
lyrics_df.head()

,artist,filename,title,lyrics
0,cher,cher_88degrees.txt,"""88 Degrees""","Stuck in L.A., ain't got no friends And so Hol..."
1,cher,cher_adifferentkindoflovesong.txt,"""A Different Kind Of Love Song""",What if the world was crazy and I was sane Wou...
2,cher,cher_afterall.txt,"""After All""","Well, here we are again I guess it must be fat..."
3,cher,cher_again.txt,"""Again""",Again evening finds me at your door Here to as...
4,cher,cher_alfie.txt,"""Alfie""","What's it all about, Alfie? Is it just for the..."


{'filename': 'robyn_youvegotthatsomething.txt',
 'title': '"You\'ve Got That Something"\n'}

In [ ]:
# Read in the twitter data
twitter_path = data_location + twitter_folder

## Data Cleaning

Now clean and tokenize your data. Remove punctuation chacters (available in the `punctuation` object in the `string` library), split on whitespace, fold to lowercase, and remove stopwords. Store your cleaned data, which must be accessible as an interable for `descriptive_stats`, in new objects or in new columns in your data frame. 



In [ ]:
punctuation = set(punctuation) # speeds up comparison

In [ ]:
# create your clean twitter data here

In [ ]:
# create your clean lyrics data here

## Basic Descriptive Statistics

Call your `descriptive_stats` function on both your lyrics data and your twitter data and for both artists (four total calls). 

In [ ]:
# calls to descriptive_stats here

Q: How do you think the "top 5 words" would be different if we left stopwords in the data? 

A: 

---

Q: What were your prior beliefs about the lexical diversity between the artists? Does the difference (or lack thereof) in lexical diversity between the artists conform to your prior beliefs? 

A: 



## Specialty Statistics

The descriptive statistics we have calculated are quite generic. You will now calculate a handful of statistics tailored to these data.

1. Ten most common emojis by artist in the twitter descriptions.
1. Ten most common hashtags by artist in the twitter descriptions.
1. Five most common words in song titles by artist. 
1. For each artist, a histogram of song lengths (in terms of number of tokens) 

We can use the `emoji` library to help us identify emojis and you have been given a function to help you.


In [6]:
assert(emoji.is_emoji("❤️"))
assert(not emoji.is_emoji(":-)"))

### Emojis 😁

What are the ten most common emojis by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Hashtags

What are the ten most common hashtags by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Song Titles

What are the five most common words in song titles by artist? The song titles should be on the first line of the lyrics pages, so if you have kept the raw file contents around, you will not need to re-read the data.


In [ ]:
# Your code here

### Song Lengths

For each artist, a histogram of song lengths (in terms of number of tokens). If you put the song lengths in a data frame with an artist column, matplotlib will make the plotting quite easy. An example is given to help you out. 


In [ ]:
num_replicates = 1000

df = pd.DataFrame({
    "artist" : ['Artist 1'] * num_replicates + ['Artist 2']*num_replicates,
    "length" : np.concatenate((np.random.poisson(125,num_replicates),np.random.poisson(150,num_replicates)))
})

df.groupby('artist')['length'].plot(kind="hist",density=True,alpha=0.5,legend=True)

Since the lyrics may be stored with carriage returns or tabs, it may be useful to have a function that can collapse whitespace, using regular expressions, and be used for splitting. 

Q: What does the regular expression `'\s+'` match on? 

A: 


In [ ]:
collapse_whitespace = re.compile(r'\s+')

def tokenize_lyrics(lyric) : 
    """strip and split on whitespace"""
    return([item.lower() for item in collapse_whitespace.split(lyric)])

In [ ]:
# Your lyric length comparison chart here. 